In [7]:
from optimize_model import optimize_model
from cobra_utils import print_logo, load_model, set_default_bounds, \
    convert_model_format, fetch_norm_sample_metabolomics_data, match_names_to_vmh

# Define paths
json_model_path = "example_data/models/microbiota_model_diet_Case_1_18_month.json"
matlab_model_path = "example_data/models/microbiota_model_diet_Case_1_18_month.mat"
metabolomics_path = "example_data/metabolomics_data.csv"
avg_eu_diet_path = "example_data/AverageEU_diet_fluxes.txt"

# Load the model
model = load_model(json_model_path)


[START] Loading model from example_data/models/microbiota_model_diet_Case_1_18_month.json...

[DONE] microbiota_model_diet_Case_1_18_month loaded.


In [10]:
fetch_norm_sample_metabolomics_data(model_input=model, gcms_filepath=metabolomics_path)

UnboundLocalError: local variable 'model' referenced before assignment

In [9]:
#########################################################
# Part 3: maximize the flux of fecal exchange reactions
#########################################################

print(f"\n[STARTED] Part 3: maximizing EX_met[fe] fluxes for {model.name}")

# Fetch all UFEt reactions and store them in a list
FEX_rxn_list = []
for rxn in model.reactions:
    if rxn.id.startswith("EX_") and rxn.id.endswith("[fe]"):
        FEX_rxn_list.append(rxn.id)
        print(f"Added {rxn.id} to the list of FEX reactions.")


[STARTED] Part 3: maximizing EX_met[fe] fluxes for microbiota_model_diet_Case_1_18_month
Added EX_12ppd_S[fe] to the list of FEX reactions.
Added EX_1hibup_S[fe] to the list of FEX reactions.
Added EX_1hibupglu_S[fe] to the list of FEX reactions.
Added EX_1hmdgluc[fe] to the list of FEX reactions.
Added EX_1ohmdz[fe] to the list of FEX reactions.
Added EX_26dap_M[fe] to the list of FEX reactions.
Added EX_2dmmq8[fe] to the list of FEX reactions.
Added EX_2hatvacid[fe] to the list of FEX reactions.
Added EX_2hatvacidgluc[fe] to the list of FEX reactions.
Added EX_2hatvlac[fe] to the list of FEX reactions.
Added EX_2hatvlacgluc[fe] to the list of FEX reactions.
Added EX_2hibup_S[fe] to the list of FEX reactions.
Added EX_2hibupglu_S[fe] to the list of FEX reactions.
Added EX_2hyoxplac[fe] to the list of FEX reactions.
Added EX_2obut[fe] to the list of FEX reactions.
Added EX_2oh_cbz[fe] to the list of FEX reactions.
Added EX_2oh_cbz_glc[fe] to the list of FEX reactions.
Added EX_2oh_mtz

In [ ]:
# Maximize the flux through all UFEt reactions
counter = 0
counter_max = len(FEX_rxn_list)
maximized_FEX_flux_list = []
for rxn in FEX_rxn_list:
    counter += 1
    print(
        f"\nMaximizing fecal exchange reaction {str(counter)} of {str(counter_max)} for {model.name}"
    )
    model.objective = rxn
    solution = model.optimize()
    maximized_FEX_flux_list.append(solution.objective_value)
    print(f"{rxn}:\t{solution.objective_value}")

# Create a dictionary of the maximized UFEt fluxes
maximized_UFEt_flux_dict = dict(zip(FEX_rxn_list, maximized_FEX_flux_list))

print(f"\n[COMPLETED] Part 1: maximization complete for {model.name}")